# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 12:00AM,238924,10,0085740674,ISDIN Corporation,Released
1,Jun 7 2022 12:00AM,238924,10,0085740683,ISDIN Corporation,Released
2,Jun 7 2022 12:00AM,238924,10,0085740682,ISDIN Corporation,Released
3,Jun 7 2022 12:00AM,238924,10,0085740684,ISDIN Corporation,Released
4,Jun 7 2022 12:00AM,238924,10,0085740704,ISDIN Corporation,Released
5,Jun 7 2022 12:00AM,238924,10,0085740703,ISDIN Corporation,Released
6,Jun 7 2022 12:00AM,238924,10,0085740706,ISDIN Corporation,Released
7,Jun 7 2022 12:00AM,238924,10,0085740707,ISDIN Corporation,Released
8,Jun 7 2022 12:00AM,238924,10,0085740710,ISDIN Corporation,Released
9,Jun 7 2022 12:00AM,238924,10,0085740708,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,238918,Released,14
35,238924,Released,44
36,238925,Released,3
37,238927,Released,6
38,238929,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238918,NaN,14.0
238924,NaN,44.0
238925,NaN,3.0
238927,NaN,6.0
238929,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238844,0.0,3.0
238852,1.0,1.0
238857,1.0,0.0
238868,0.0,1.0
238869,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238844,0,3
238852,1,1
238857,1,0
238868,0,1
238869,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238844,0,3
1,238852,1,1
2,238857,1,0
3,238868,0,1
4,238869,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238844,,3
1,238852,1,1
2,238857,1,
3,238868,,1
4,238869,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation
21,Jun 7 2022 12:00AM,238925,10,ISDIN Corporation
26,Jun 7 2022 12:00AM,238927,10,Emerginnova
53,Jun 6 2022 3:51PM,238929,20,Alumier Labs Inc.
54,Jun 6 2022 3:20PM,238917,20,Alumier Labs Inc.
108,Jun 6 2022 3:18PM,238918,10,ISDIN Corporation
122,Jun 6 2022 3:06PM,238916,21,"NBTY Global, Inc."
123,Jun 6 2022 2:58PM,238915,19,ACG North America LLC
124,Jun 6 2022 2:57PM,238914,20,Reliable 1 Laboratories LLC
125,Jun 6 2022 2:54PM,238913,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation,,44
1,Jun 7 2022 12:00AM,238925,10,ISDIN Corporation,,3
2,Jun 7 2022 12:00AM,238927,10,Emerginnova,,6
3,Jun 6 2022 3:51PM,238929,20,Alumier Labs Inc.,,1
4,Jun 6 2022 3:20PM,238917,20,Alumier Labs Inc.,,54
5,Jun 6 2022 3:18PM,238918,10,ISDIN Corporation,,14
6,Jun 6 2022 3:06PM,238916,21,"NBTY Global, Inc.",,1
7,Jun 6 2022 2:58PM,238915,19,ACG North America LLC,,1
8,Jun 6 2022 2:57PM,238914,20,Reliable 1 Laboratories LLC,1,
9,Jun 6 2022 2:54PM,238913,10,Yusen – 3D Matrix,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation,44,
1,Jun 7 2022 12:00AM,238925,10,ISDIN Corporation,3,
2,Jun 7 2022 12:00AM,238927,10,Emerginnova,6,
3,Jun 6 2022 3:51PM,238929,20,Alumier Labs Inc.,1,
4,Jun 6 2022 3:20PM,238917,20,Alumier Labs Inc.,54,
5,Jun 6 2022 3:18PM,238918,10,ISDIN Corporation,14,
6,Jun 6 2022 3:06PM,238916,21,"NBTY Global, Inc.",1,
7,Jun 6 2022 2:58PM,238915,19,ACG North America LLC,1,
8,Jun 6 2022 2:57PM,238914,20,Reliable 1 Laboratories LLC,,1
9,Jun 6 2022 2:54PM,238913,10,Yusen – 3D Matrix,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation,44,
1,Jun 7 2022 12:00AM,238925,10,ISDIN Corporation,3,
2,Jun 7 2022 12:00AM,238927,10,Emerginnova,6,
3,Jun 6 2022 3:51PM,238929,20,Alumier Labs Inc.,1,
4,Jun 6 2022 3:20PM,238917,20,Alumier Labs Inc.,54,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation,44.0,NaN
1,Jun 7 2022 12:00AM,238925,10,ISDIN Corporation,3.0,NaN
2,Jun 7 2022 12:00AM,238927,10,Emerginnova,6.0,NaN
3,Jun 6 2022 3:51PM,238929,20,Alumier Labs Inc.,1.0,NaN
4,Jun 6 2022 3:20PM,238917,20,Alumier Labs Inc.,54.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation,44.0,0.0
1,Jun 7 2022 12:00AM,238925,10,ISDIN Corporation,3.0,0.0
2,Jun 7 2022 12:00AM,238927,10,Emerginnova,6.0,0.0
3,Jun 6 2022 3:51PM,238929,20,Alumier Labs Inc.,1.0,0.0
4,Jun 6 2022 3:20PM,238917,20,Alumier Labs Inc.,54.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2627934,78.0,0.0
12,238886,1.0,0.0
15,238852,1.0,1.0
16,477814,12.0,14.0
17,477784,2.0,0.0
18,477771,2.0,0.0
19,1194485,22.0,0.0
20,1672259,127.0,57.0
21,955615,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2627934,78.0,0.0
1,12,238886,1.0,0.0
2,15,238852,1.0,1.0
3,16,477814,12.0,14.0
4,17,477784,2.0,0.0
5,18,477771,2.0,0.0
6,19,1194485,22.0,0.0
7,20,1672259,127.0,57.0
8,21,955615,4.0,0.0
9,22,238869,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,78.0,0.0
1,12,1.0,0.0
2,15,1.0,1.0
3,16,12.0,14.0
4,17,2.0,0.0
5,18,2.0,0.0
6,19,22.0,0.0
7,20,127.0,57.0
8,21,4.0,0.0
9,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,78.0
1,12,Released,1.0
2,15,Released,1.0
3,16,Released,12.0
4,17,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Executing,0.0,0.0,1.0,14.0,0.0,0.0,0.0,57.0,0.0,0.0
Released,78.0,1.0,1.0,12.0,2.0,2.0,22.0,127.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Executing,0.0,0.0,1.0,14.0,0.0,0.0,0.0,57.0,0.0,0.0
1,Released,78.0,1.0,1.0,12.0,2.0,2.0,22.0,127.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Executing,0.0,0.0,1.0,14.0,0.0,0.0,0.0,57.0,0.0,0.0
1,Released,78.0,1.0,1.0,12.0,2.0,2.0,22.0,127.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()